In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import hsfm

## Pre-select NAGAP imagery
- The nagap_image_metadata.csv file has been compiled from NAGAP metadata files available at www.arcticdata.io. 
- The file contains all images for which a pid_tiff (tif image) ID is available in the metadata. 
- Some images do not have lat lon positional information, but are still included in the csv. 
- Setting lat lon bounds in the function below will remove those values, while only specifying a year, for example, will return them in the dataframe (if desired).
- Specify target bounds as (ULLON, ULLAT, LRLON, LRLAT)
- Specify target year as 77, e.g. for year 1977

In this example we specify bounds to examine Easton Glacier at Mt Baker, WA for images taken in 1977.

In [ ]:
nagap_metadata_csv = 'input_data/nagap_image_metadata.csv'

In [ ]:
bounds= (-121.846, 48.76, -121.823, 48.70) # approximate bounds for Easton glacier
year = 77

In [ ]:
df = hsfm.core.pre_select_NAGAP_images(nagap_metadata_csv,
                                       bounds = bounds,
                                       year = year)
df

## Download SRTM reference DEM
- Note that the images above do not contain altitude values in the metadata. 
- We can download a coarse SRTM reference DEM to extract approximate flight altitudes.
- We specify broader bounds here to cover Mt Baker, again as (ULLON, ULLAT, LRLON, LRLAT).

In [ ]:
bounds = (-121.90, 48.85, -121.60, 48.65) # approximate bounds for Mt Baker

In [ ]:
srtm_reference_dem = hsfm.utils.download_srtm(bounds)

In [ ]:
srtm_reference_dem = './input_data/reference_dem/srtm_subset-adj.tif'
hsfm.plot.plot_dem_from_file(srtm_reference_dem)

## Pre-process the imagery
- Download thumbnails to disk to read off focal length

In [ ]:
hsfm.batch.download_images_to_disk(df, 
                                   output_directory='input_data/thumbnails',
                                   image_type='pid_tn') # pid_tiff, pid_tn, pid_jpeg

In [ ]:
focal_length = 151.303 # read off image frame in downloaded thumbnails

- Detect fiducial markers, crop, and enhance contrast

In [ ]:
template_directory = 'input_data/fiducials/nagap/notch'

In [ ]:
hsfm.batch.preprocess_images(template_directory,
                             image_metadata=df,
                             output_directory='input_data/processed_images',
                             qc=True)

## Determine image clusters
- This step determines if there are multiple clusters of images present within the selection.
- If there is no overlap between clusters of images, they must be processed into seperate DEMs for alignment in later steps to be successful.
- In this example, there is only one cluster of images.

In [ ]:
hsfm.core.determine_image_clusters(df,
                                   output_directory='input_data',
                                   reference_dem=srtm_reference_dem,
                                   image_directory = 'input_data/processed_images')

## Download high resolution reference dem

In [ ]:
reference_dem = 'input_data/reference_dem/lidar/baker.tif'

In [ ]:
import driveanon as da
!mkdir input_data/reference_dem/lidar/
da.save('1ObQyjhYB_fjhvqtBq-vK3CdPoQ1Iauyd', filename=reference_dem)

## Run processing with Metashape

In [ ]:
# hsfm.metashape.authentication('path/to/licence/file.lic')

In [ ]:
image_matching_accuracy = 4
densecloud_quality      = 4

In [ ]:
project_name          = 'easton'
output_path           = 'metashape/'
images_path           = 'input_data/cluster_000/images'
images_metadata_file  = 'input_data/cluster_000/metashape_metadata.csv'
focal_length          = 151.303
pixel_pitch           = 0.02
verbose               = True
rotation_enabled      = True

In [ ]:
project_file, point_cloud_file = hsfm.metashape.images2las(project_name,
                                            images_path,
                                            images_metadata_file,
                                            output_path,
                                            focal_length            = focal_length,
                                            pixel_pitch             = pixel_pitch,
                                            image_matching_accuracy = image_matching_accuracy,
                                            densecloud_quality      = densecloud_quality,
                                            rotation_enabled        = rotation_enabled)

In [ ]:
epsg_code = 'EPSG:'+ hsfm.geospatial.get_epsg_code(reference_dem)
dem = hsfm.asp.point2dem(point_cloud_file, 
                         '--nodata-value','-9999',
                         '--tr','0.5',
                         '--threads', '10',
                         '--t_srs', epsg_code,
                         verbose=verbose)

In [ ]:
clipped_reference_dem = 'metashape/reference_dem_clip.tif'
clipped_reference_dem = hsfm.utils.clip_reference_dem(dem, 
                                                      reference_dem,
                                                      output_file_name = clipped_reference_dem,
                                                      buff_size        = 2000,
                                                      verbose = verbose)

In [ ]:
aligned_dem_file, _ =  hsfm.asp.pc_align_p2p_sp2p(dem,
                                                  clipped_reference_dem,
                                                  output_path,
                                                  verbose = verbose)

In [ ]:
hsfm.utils.dem_align_custom(clipped_reference_dem,
                            aligned_dem_file,
                            output_path,
                            verbose = verbose)